In [1]:
!ls
!pwd

00-erisk25task1EDA.ipynb     04-eRiskAnalysisSelfReferential.ipynb
01-erisktokenestimate.ipynb  05-eRiskPySparkSelfRefFiltering.ipynb
03-pyterrier-test2.ipynb     06-eRiskPySparkSelfRefSpacy.ipynb
/storage/home/hcoda1/6/dahumada3/clef/erisk-2025/user/dahumada3/notebooks


In [2]:
import sys

sys.path.append("/storage/home/hcoda1/6/dahumada3/clef/erisk-2025")

from erisk.spark import get_spark

spark = get_spark()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 01:58:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/28 01:58:12 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# Set paths
LABELS_MAJ_PATH = "/storage/home/hcoda1/6/dahumada3/erisk_shared/raw/training_data/2023/g_qrels_majority_2.csv"
LABELS_CONS_PATH = "/storage/home/hcoda1/6/dahumada3/erisk_shared/raw/training_data/2023/g_rels_consenso.csv"
PARQUET_DIR = "/storage/home/hcoda1/6/dahumada3/erisk_shared/parquet/training_data/2023/partitions"

In [4]:
df = spark.read.parquet(PARQUET_DIR)
df.printSchema()
df.show(5, truncate=False)


[Stage 0:>                                                          (0 + 1) / 1]



root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)




[Stage 2:============================================>              (3 + 1) / 4]



+------------+-------------------------------------------------------------------------------------+
|DOCNO       |TEXT                                                                                 |
+------------+-------------------------------------------------------------------------------------+
|s_2457_109_0|LADWP, Inyo agree to test run on well 395                                            |
|s_2457_110_0|State representatives recognize NIHDs District of Year designation                   |
|s_2457_111_0|Small plane crashes en route from Bishop to Nanaimo, BC (Canada)                     |
|s_2457_112_0|Audio and video production professionals, as an example: https://youtu.be/jv5HIrOrn2o|
|s_2457_113_0|Sure, its a professional powerhouse for audio and video production professionals.    |
+------------+-------------------------------------------------------------------------------------+
only showing top 5 rows



In [5]:
# Dataframe cleanup
from pyspark.sql.functions import col, trim
from pyspark.sql.functions import regexp_replace

df_clean = df.filter(trim(col("TEXT")) != "")
df_clean = df_clean.withColumn("TEXT", regexp_replace("TEXT", r"http\S+|www\S+", ""))
df_clean.select("TEXT").show(5, truncate=False)

print("Remaining rows:", df_clean.count())

+---------------------------------------------------------------------------------+
|TEXT                                                                             |
+---------------------------------------------------------------------------------+
|LADWP, Inyo agree to test run on well 395                                        |
|State representatives recognize NIHDs District of Year designation               |
|Small plane crashes en route from Bishop to Nanaimo, BC (Canada)                 |
|Audio and video production professionals, as an example:                         |
|Sure, its a professional powerhouse for audio and video production professionals.|
+---------------------------------------------------------------------------------+
only showing top 5 rows



Remaining rows: 4264560


In [6]:
from pyspark.sql.functions import lower

SELF_REF_PATTERN = r"\b(i|me|my|mine|myself|i'm|i’ve|i'd|i’ll|i am|i was)\b"

df_with_flag = df_clean.withColumn(
    "is_self_ref", lower(col("TEXT")).rlike(SELF_REF_PATTERN)
)

In [7]:
self_ref_df = df_with_flag.filter(col("is_self_ref"))

total_count = df_with_flag.count()
self_ref_count = self_ref_df.count()

print(f"Self-referential posts: {self_ref_count:,} out of {total_count:,}")

Self-referential posts: 1,191,200 out of 4,264,560


In [8]:
df_with_flag.show(5)

+------------+--------------------+-----------+
|       DOCNO|                TEXT|is_self_ref|
+------------+--------------------+-----------+
|s_2457_109_0|LADWP, Inyo agree...|      false|
|s_2457_110_0|State representat...|      false|
|s_2457_111_0|Small plane crash...|      false|
|s_2457_112_0|Audio and video p...|      false|
|s_2457_113_0|Sure, its a profe...|      false|
+------------+--------------------+-----------+
only showing top 5 rows



In [9]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
SELF_REF_WORDS = {
    "i",
    "me",
    "my",
    "mine",
    "myself",
    "i'm",
    "i’ve",
    "i'd",
    "i’ll",
    "i am",
    "i was",
}


def spacy_self_ref_ratio(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_space]
    if not tokens:
        return 0.0
    self_ref_count = sum(1 for token in tokens if token in SELF_REF_WORDS)
    return self_ref_count / len(tokens)

In [10]:
from pyspark.sql import Row


def process_partition_with_spacy(partition):
    import spacy

    nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
    SELF_REF_WORDS = {
        "i",
        "me",
        "my",
        "mine",
        "myself",
        "i'm",
        "i’ve",
        "i'd",
        "i’ll",
        "i am",
        "i was",
    }

    for row in partition:
        text = row["TEXT"]
        doc = nlp(text.lower())
        tokens = [token.text for token in doc if not token.is_space]
        if not tokens:
            ratio = 0.0
        else:
            self_ref_count = sum(1 for token in tokens if token in SELF_REF_WORDS)
            ratio = self_ref_count / len(tokens)

        yield Row(**row.asDict(), self_ref_ratio=ratio)

In [11]:
import time

start = time.time()

# Your Spark transformation
df_with_ratio = df_with_flag.rdd.mapPartitions(process_partition_with_spacy).toDF()

end = time.time()
print(f"⏱️ Completed in {end - start:.2f} seconds")

⏱️ Completed in 36.03 seconds


In [12]:
df_with_ratio.show(5)

+------------+--------------------+-----------+--------------+
|       DOCNO|                TEXT|is_self_ref|self_ref_ratio|
+------------+--------------------+-----------+--------------+
|s_2457_109_0|LADWP, Inyo agree...|      false|           0.0|
|s_2457_110_0|State representat...|      false|           0.0|
|s_2457_111_0|Small plane crash...|      false|           0.0|
|s_2457_112_0|Audio and video p...|      false|           0.0|
|s_2457_113_0|Sure, its a profe...|      false|           0.0|
+------------+--------------------+-----------+--------------+
only showing top 5 rows



25/03/28 02:03:14 WARN PythonRunner: Detected deadlock while completing task 2.0 in stage 19 (TID 98): Attempting to kill Python Worker



In [17]:
df_with_ratio.select("TEXT", "is_self_ref", "self_ref_ratio").show(10, truncate=False)

+----------------------------------------------------------------------------------------+-----------+-------------------+
|TEXT                                                                                    |is_self_ref|self_ref_ratio     |
+----------------------------------------------------------------------------------------+-----------+-------------------+
|LADWP, Inyo agree to test run on well 395                                               |false      |0.0                |
|State representatives recognize NIHDs District of Year designation                      |false      |0.0                |
|Small plane crashes en route from Bishop to Nanaimo, BC (Canada)                        |false      |0.0                |
|Audio and video production professionals, as an example:                                |false      |0.0                |
|Sure, its a professional powerhouse for audio and video production professionals.       |false      |0.0                |
|I have no use f

In [ ]:
df_with_ratio.filter((col("self_ref_ratio") > 0) & (~col("is_self_ref"))).show(
    5, truncate=False
)

+------------+---------------------------------------------------------------------------------------------------------------------------+-----------+--------------------+
|DOCNO       |TEXT                                                                                                                       |is_self_ref|self_ref_ratio      |
+------------+---------------------------------------------------------------------------------------------------------------------------+-----------+--------------------+
|s_2457_120_0|Thanks, Im really looking forward to using the new gear.                                                                   |false      |0.07692307692307693 |
|s_2457_120_1|The only piece Im nervous about is the Sawyer Micro.                                                                       |false      |0.08333333333333333 |
|s_2457_120_2|Ive watched some YouTube video reviews complaining that they tend to block up with just routine use.                       |fa

In [ ]:
df_with_ratio.filter((col("is_self_ref")) & (~col("self_ref_ratio"))).show(
    5, truncate=False
)

+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------+
|DOCNO       |TEXT                                                                                                                                                                       |is_self_ref|self_ref_ratio|
+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------+
|s_2457_131_0| gt;I can imagine that the forces are far greater - a 3ft long blade (from root to tip) on an aircraft is nothing compared to the 300+ foot long monsters on wind turbines.|true       |0.0           |
|s_3020_36_0 |[Translation](  gt;my method of healing as of late!!!                                                                             

25/03/28 02:26:34 WARN PythonRunner: Detected deadlock while completing task 2.0 in stage 25 (TID 113): Attempting to kill Python Worker

